In [4]:
# imports
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (
    Embedding,
    Dense,
    LSTM,
    Dropout,
    Bidirectional,
)


In [1]:
# Deep learning: 

class RnnModel():

    def __init__(self, embedding_matrix, embedding_dim, max_len, X_additional=None, label = ""):
        
        inp1 = Input(shape=(max_len,))
        x = Embedding(embedding_matrix.shape[0], embedding_dim, weights=[embedding_matrix])(inp1)
        x = Bidirectional(LSTM(256, return_sequences=True))(x)
        x = Bidirectional(LSTM(150))(x)
        x = Dense(128, activation="relu")(x)
        x = Dropout(0.1)(x)
        x = Dense(64, activation="relu")(x)
        x = Dense(1, activation="sigmoid")(x)    
        model = Model(inputs=inp1, outputs=x)
        # bidirectional
        # mit 1 -> 'mse'
        # mit 6 -> 'binary_crossentropy', softmax statt sigmoid
        model.compile(loss = 'binary_crossentropy', optimizer = 'adam')
        self.model = model
        self.label = label

In [6]:
class all_party_train:
    def __init__(
        self,
        X_train: list,
        Y_train: list,
        embed_matrix,
        max_len,
        labels,
        X_test=[],
        Y_test=[],
        epochs=3,
        batch_size=1024,
    ):

        self.models = []
        self.yhat_list = []
        self.acc_list = []

        for number, label in enumerate(labels):
            # Creating the model
            rnn = RnnModel(
                embedding_matrix=embed_matrix,
                embedding_dim=len(embed_matrix[0]),
                max_len=max_len,
                label=label,
            )
            # ajust label to current party
            Y = np.array([1 if y_t == number else 0 for y_t in Y_train], np.int8)
            rnn.model.fit(X_train, Y, batch_size=batch_size, epochs=epochs)
            self.models.append(rnn.model)

        if len(X_test) > 0:
            for number, model in enumerate(self.models): 
                yhat = [x[0] for x in model.predict(X_test).tolist()]
                self.yhat_list.append(yhat)

                # If true labels are provided we calculate the accuracy of the model
                if len(Y_test) > 0:
                    ytest = np.array([1 if y_te == number else 0 for y_te in Y_test])
                    acc = accuracy_score(ytest, [1 if x > 0.5 else 0 for x in yhat])
                    self.acc_list.append(acc)